In [2]:
import pandas as pd

import numpy as np

import datetime

import sys 

In [5]:
filepath="datasets\S50F_M15.csv"

chunksize_val=5000
chunkDFList=[]
for df_chunk in pd.read_csv(filepath,chunksize=chunksize_val):

 #f_chunk.drop(columns='Ticker',inplace=True)
 
 #f_chunk.rename(columns={'Date/Time':'datetime'},inplace=True)
 
 df_chunk['datetime']=pd.to_datetime(df_chunk["datetime"],format='%d/%m/%Y %H:%M:%S',dayfirst=True)
 
 df_chunk['x_date'] = [d.date() for d in df_chunk['datetime']] 
# get date from datetime as object using for
 df_chunk['x_date']=pd.to_datetime(df_chunk["x_date"],format='%Y-%m-%d')
# convert object to datetime
 
 df_chunk['x_time'] = pd.to_datetime(df_chunk['datetime'],format='%Y-%m-%d %H:%M:%S').dt.time   
#get date from datetime as object pd.datetime from 'datetime
 df_chunk['x_time']=pd.to_datetime(df_chunk['x_time'],format='%H:%M:%S')
#convert time to datetime type

 df_chunk['x_weekdayNo'] = df_chunk['datetime'].dt.strftime('%w').astype(int)
 df_chunk['x_weekdayName'] = df_chunk['datetime'].dt.strftime('%A')
 chunkDFList.append(df_chunk)

 df_chunk['x_noBar']=1
    
df_x=pd.concat(chunkDFList,axis=0)



In [6]:
original_totalrow=df_x.shape[0]
print(f'Data before removing = {original_totalrow}')
print(df_x.info())
df_x.sample(10)

Data before removing = 72653
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72653 entries, 0 to 72652
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   datetime       72653 non-null  datetime64[ns]
 1   open           72653 non-null  float64       
 2   high           72653 non-null  float64       
 3   low            72653 non-null  float64       
 4   close          72653 non-null  float64       
 5   Volume         72653 non-null  float64       
 6   x_date         72653 non-null  datetime64[ns]
 7   x_time         72653 non-null  datetime64[ns]
 8   x_weekdayNo    72653 non-null  int32         
 9   x_weekdayName  72653 non-null  object        
 10  x_noBar        72653 non-null  int64         
dtypes: datetime64[ns](3), float64(5), int32(1), int64(1), object(1)
memory usage: 5.8+ MB
None


,datetime,open,high,low,close,Volume,x_date,x_time,x_weekdayNo,x_weekdayName,x_noBar
68932,2019-07-26 11:15:00,1130.0,1130.3,1128.8,1129.1,265.0,2019-07-26,1900-01-01 11:15:00,5,Friday,1
11126,2008-08-01 10:30:00,458.7,461.6,458.7,461.6,272.0,2008-08-01,1900-01-01 10:30:00,5,Friday,1
37934,2013-10-24 14:30:00,992.0,996.0,991.5,992.2,1000.0,2013-10-24,1900-01-01 14:30:00,4,Thursday,1
12508,2008-11-05 10:00:00,330.8,333.3,329.0,331.7,1106.0,2008-11-05,1900-01-01 10:00:00,3,Wednesday,1
67116,2019-03-25 16:30:00,1078.4,1081.9,1078.4,1081.7,547.0,2019-03-25,1900-01-01 16:30:00,1,Monday,1
69303,2019-08-22 10:15:00,1067.8,1068.9,1066.1,1068.5,781.0,2019-08-22,1900-01-01 10:15:00,4,Thursday,1
36983,2013-08-21 11:00:00,917.5,919.5,916.0,916.6,1160.0,2013-08-21,1900-01-01 11:00:00,3,Wednesday,1
13613,2009-01-23 15:15:00,292.9,293.2,291.6,291.7,115.0,2009-01-23,1900-01-01 15:15:00,5,Friday,1
59756,2017-11-14 16:45:00,1085.1,1086.0,1085.0,1085.7,3201.0,2017-11-14,1900-01-01 16:45:00,2,Tuesday,1
27540,2011-10-19 12:00:00,657.4,657.4,654.7,656.0,687.0,2011-10-19,1900-01-01 12:00:00,3,Wednesday,1


# Remove Weekend

In [7]:
df_x['x_weekdayName'].value_counts()
#df_x['x_weekdayNo'].value_counts()

Thursday     14870
Wednesday    14859
Friday       14682
Tuesday      14613
Monday       13629
Name: x_weekdayName, dtype: int64

In [8]:
print("Not Day of Trade  : Before removing: ",df_x.shape)
df_x.drop(index=(df_x[df_x['x_weekdayNo'].isin([0,6])].index),inplace=True)
print("Not Day of Trade  : After removed: ",df_x.shape)

Not Day of Trade  : Before removing:  (72653, 11)
Not Day of Trade  : After removed:  (72653, 11)


# Remove Not Time of Trade

In [9]:
df_x[ (df_x['x_time'].dt.strftime('%H:%M:%S')<'09:45:00') | (df_x['x_time'].dt.strftime('%H:%M:%S')>'17:00:00') ]

,datetime,open,high,low,close,Volume,x_date,x_time,x_weekdayNo,x_weekdayName,x_noBar
68508,2019-07-01 09:41:50,1161.9,1162.0,1159.3,1161.0,0.0,2019-07-01,1900-01-01 09:41:50,1,Monday,1
68532,2019-07-02 09:30:00,1153.5,1153.5,1153.5,1153.5,0.0,2019-07-02,1900-01-01 09:30:00,2,Tuesday,1
68558,2019-07-03 09:30:00,1150.0,1150.0,1149.9,1149.9,0.0,2019-07-03,1900-01-01 09:30:00,3,Wednesday,1
68583,2019-07-04 09:30:00,1153.2,1153.3,1153.2,1153.2,0.0,2019-07-04,1900-01-01 09:30:00,4,Thursday,1
68608,2019-07-05 09:30:00,1142.2,1143.0,1142.2,1142.7,0.0,2019-07-05,1900-01-01 09:30:00,5,Friday,1


In [10]:
print("Not Time of Trade  : Before removing: ",df_x.shape)
df_x=df_x[df_x['x_time'].dt.strftime('%H:%M:%S').between('09:45:00','17:00:00')]

print("Not Time of Trade  : After removed: ",df_x.shape)

Not Time of Trade  : Before removing:  (72653, 11)
Not Time of Trade  : After removed:  (72648, 11)


# Remove Holiday 

In [11]:
invalidDay_nBars=18

In [12]:
df_GbyNoBar=df_x.groupby(['x_date'])[['x_noBar']].count()
df_GbyNoBar.reset_index(inplace=True)


In [13]:
invalidDate=df_GbyNoBar.query('x_noBar<=@invalidDay_nBars')
invalidDate

,x_date,x_noBar
0,2006-06-02,15
1,2006-06-05,18
2,2006-06-06,15
7,2006-06-15,18
795,2009-09-02,17
806,2009-09-17,15
966,2010-05-17,15
967,2010-05-18,15
968,2010-05-19,11
1818,2013-11-06,10


In [14]:
invalidDate_idx=df_x[ df_x.x_date.isin( df_GbyNoBar.query('x_noBar<=@invalidDay_nBars')['x_date'].tolist())].index
print("Invali Date of Trade  : Before removing: ",df_x.shape)
df_x.drop(index=invalidDate_idx,inplace=True)
print("Invali Date of Trade  : After removed: ",df_x.shape)

Invali Date of Trade  : Before removing:  (72648, 11)
Invali Date of Trade  : After removed:  (72482, 11)


In [15]:
last_totalrow=df_x.shape[0]
print(last_totalrow)

72482



df_x[ df_x['x_date']=="2009-09-17"]

In [16]:
col_remove=['x_date','x_time','x_weekdayNo','x_weekdayName','x_noBar']
df_x.drop(columns=col_remove,inplace=True)

df_x.set_index('datetime',inplace=True)

In [17]:
df_x.head(5)

,open,high,low,close,Volume
datetime,,,,,
2006-06-07 09:45:00,486.0,487.0,483.5,484.8,7.0
2006-06-07 10:00:00,484.1,484.1,482.7,483.9,18.0
2006-06-07 10:15:00,483.0,483.0,482.6,483.0,14.0
2006-06-07 10:30:00,483.1,483.1,483.0,483.0,6.0
2006-06-07 10:45:00,483.2,483.2,483.2,483.2,3.0


In [18]:
df_x.tail(5)

,open,high,low,close,Volume
datetime,,,,,
2020-03-31 15:45:00,746.6,746.7,741.8,742.4,1302.0
2020-03-31 16:00:00,742.1,744.9,742.0,743.0,774.0
2020-03-31 16:15:00,743.1,747.8,743.0,745.7,809.0
2020-03-31 16:30:00,745.8,746.5,742.5,743.8,724.0
2020-03-31 16:45:00,743.9,743.9,742.1,743.5,563.0


In [19]:
print(f'Data after removing = {last_totalrow}')
print(f'Data after remove = {round(last_totalrow/original_totalrow,4)*100}% of all')

Data after removing = 72482
Data after remove = 99.76% of all


In [20]:
df_x.to_csv('S50IF_CON.csv')